# Object classification model using SIFT features

This notebook has two sections. Section 1 uses full images and extracts bounding boxes with manual coordinates and processes throug a SIFT histogram model. SVM and Logistic regression are used on train and validation sets for classfication. There is no data pre-processing or augmentation in this section. Section 2 uses already pre-processed and augmented bounding boxes as input data for training the SIFT model. This section also uses SVM and Logistic regression for classification.

In [ ]:
# Install requied libraries
!python -m pip install --upgrade pip
!python -m pip install opencv-contrib-python --user

In [1]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import io
import json
import pandas as pd
import pickle

from PIL import Image
from sklearn.cluster import KMeans
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from scipy.spatial import distance

In [4]:
# Set pwd
PWD = !pwd
PWD[0]

'/Users/vish/281/project'

In [7]:
# Set base dir
base_dir = PWD[0]

In [2]:
!pwd

/Users/vish/281/project


In [10]:
# Set the directory where all the pre-processed and augmented data is present
gray_dir = base_dir + "/gray/"
gray_dir

'/Users/vish/281/project/gray/'

In [5]:
#Define paths for full images and yolo bb files
image_dir = base_dir + "data/all_images/"
bbox_dir = base_dir + "data/yolofinalset/obj_train_data/"

#Change current directory
os.chdir(base_dir)

In [793]:
# Load classes and input images from json files
with open(base_dir+"classes.json") as infile:
    classes = json.load(infile)

with open(base_dir+"train_test_split_img.json") as infile:
    inp_imgs = json.load(infile)

# Check keys of train_test_split_img.json
inp_imgs.keys()

dict_keys(['X_train', 'y_train', 'X_val', 'y_val', 'X_test', 'y_test'])

### Section 1: Extract BB from images and no data augmentation or processing

In [11]:
# Function to extract a bounding box from the image 
# based on the yolo text file data

def extract_bbox(img, center_x, center_y, width, height):
    
    img_y, img_x = img.shape
    
    min_x = round((center_x - 0.5*width) * img_x)
    max_x = round((center_x + 0.5*width) * img_x)
    min_y = round((center_y - 0.5*height) * img_y)
    max_y = round((center_y + 0.5*height) * img_y)
    
    # return extracted bb
    return img[min_y:max_y, min_x:max_x]

In [795]:
# Function to extract all bounding boxes from a set of input images

def extract_all_bb(input_imgs, set_tag):
    """parameters: 
        input_imgs: input list of image names
        set_tag: "train" or "val" or "test"
        output: total bb count for input list 
    """
    # Initialize bb count for input set
    bb_cnt = 0

    # for each image get the yolo bb text file
    for image in input_imgs:
        img_path = image_dir + image
        bb_path = bbox_dir + image.replace('.jpg','.txt')


        # Convert ref image to grayscale
        img_color = plt.imread(img_path)
        img_gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
        
        # Open each yolo text and read all lines
        # yolov5 format - class_id center_x center_y width height
        with open(bb_path) as f:
            input_bb = f.readlines()


        # Assign values from yolo text file and extract bounding box
        for bb in range(len(input_bb)):
            cls, center_x, center_y, width, height = [float(x) for x in input_bb[bb].split()]
            
            #randomly generate bb variable name and assign extracted bb
            globals()['bb_%s_%s' % (set_tag, bb_cnt)] = [extract_bbox(img_gray, center_x, center_y, width, height),cls]
            
            # increment bb count
            bb_cnt += 1
    
    return bb_cnt

### Data assignment
* Get the list of tran, validation and test images from the input json
* Extract and store all bbs from the different input list
* The extrcted bbs and classes are stored in the global dictionary as a list [bb, cls]
* Variable format is bb_x_y where x is "train" or "val" or "test" ad y is the bb number in the list 

In [796]:
# Get the list of images
train_imgs = inp_imgs["X_train"]
val_imgs = inp_imgs["X_val"]
test_imgs = inp_imgs["X_test"]

# save bb and class for all sets
train_bbs = extract_all_bb(train_imgs, "train")
val_bbs = extract_all_bb(val_imgs, "val")
test_bbs = extract_all_bb(test_imgs, "test")

### Extract SIFT features from the training data and create histograms

In [797]:
# create SIFT feature extractor
sift = cv2.SIFT_create()

In [798]:
# function to get keypoints and descripors for the given list of bbs

def key_desc(bb_cnt, set_tag):
    """parameters: 
        bb_cnt: count of the input bbs
        set_tag: "train" or "val" or "test" 
    """
    
    # detect all keypoints, descriptors from all bbs
    for bb in range(bb_cnt):
        
        # Assign the keypoints and descriptors to predefined global variables using set tag
        globals()['kp_%s_%s' % (set_tag, bb)], globals()['dc_%s_%s' % (set_tag, bb)] \
                                = sift.detectAndCompute(globals()['bb_%s_%s' % (set_tag, bb)][0], None)
        
        # Assign classes to each bb to predefined global variables using set tag
        globals()['cl_%s_%s' % (set_tag, bb)] = int(globals()['bb_%s_%s' % (set_tag, bb)][1])
    
    return

Below we extract all keypoints and descriptors for the three different bb lists

In [799]:
# get all keypoints and descriptors
key_desc(train_bbs, "train")
key_desc(val_bbs, "val")
key_desc(test_bbs, "test")

Now that we have extracted all the keypoints and descriptors from the training bb list, we need to stack them up. This will help us run k-means on it and cluster them into different groups.

In [805]:
# function to stack all keypoint descriptors in a set

def dc_stack(bb_cnt, set_tag):
    """parameters: 
        bb_cnt: count of the input bbs
        set_tag: "train" or "val" or "test" 
        output: stacked descriptors (Nx128)
    """
        
    #stack all keypoint descriptors for using k-means later
    stack_dc = globals()['dc_%s_0' % (set_tag)]
    
    # Use try except for handling bbs with no keypoints or descriptors
    for i in range((bb_cnt)-1):
        try:
            stack_dc = np.vstack((stack_dc,globals()['dc_%s_%s' % (set_tag, i+1)]))
        except:
            pass
        
    return stack_dc

In [808]:
# stack all training keypoint descriptors by calling the function
train_dc = dc_stack(train_bbs, "train")

# check shape of stacked descriptors
print(train_dc.shape)

(191977, 128)


### K-means clustering
Run k-means algorithm on the extracted descriptors from the training set to form different clusters. Number of clusters is a hyper parameter arrived at after tuning.

In [809]:
# No of clusters/bins
dim = 350

# Instantiate k-means
kmeans = KMeans(init="k-means++", n_clusters=dim, n_init=4)

# Fit all training keypoints and transform
dc_km = kmeans.fit_transform(train_dc)

# Get cluster labels for the training keypoints
dc_kmy = kmeans.fit_predict(train_dc)

**Data validation - Perform some checks**

In [810]:
# Checks shapes of the transformed data
print("Shape of keypoint descriptors after clustering: ",dc_km.shape)
print("Shape of keypoint descriptor labels: ",dc_kmy.shape)

# min and max clusters
print("Minimum cluster #", min(dc_kmy), "Maximum cluster #", max(dc_kmy))

# Cluster identification for each keypoint

# Cluster centers shape
print("Cluster centers shape: ", kmeans.cluster_centers_.shape)

Shape of keypoint descriptors after clustering:  (191977, 350)
Shape of keypoint descriptor labels:  (191977,)
Minimum cluster # 0 Maximum cluster # 349
Cluster centers shape:  (350, 128)


**Building SIFT histogram for each bb based on the keypoint descriptor clusters**

In [811]:
# Function to build SIFT histogram from keypoint descriptors by grouping into clusters
def build_hist(bb_cnt, set_tag, dim):
    """parameters: 
        bb_cnt: count of the input bbs
        set_tag: "train" or "val" or "test"
        dim: number of clusters used as bins
    """
    # Use try except for handling bbs with no keypoints or descriptors
    for bb in range(bb_cnt): 
        try:
            # Run the bb keyponts through training kmeans 
            globals()['clst_%s_%s' % (set_tag, bb)] = kmeans.predict(globals()['dc_%s_%s' % (set_tag, bb)])
        except:
            # Set histogram to zero if there are no keypoints
            globals()['his_%s_%s' % (set_tag, bb)] = (np.zeros((dim,),dtype=int),np.arange(0,dim+1))
        else:
            # Create histogram based on assigned clusters when descriptors are available
            globals()['his_%s_%s' % (set_tag, bb)] = np.histogram(globals()['clst_%s_%s' % (set_tag, bb)], \
                                                                  bins=dim,range=(0,dim))
    
    return

**Building SIFT histograms for the training set**
<br/>The histogram is stored in a global variable of format is his_x_y where x is "train" or "val" or "test" ad y is the bb number in the list.

In [812]:
# Build SIFT histograms using clusters dimensions
build_hist(train_bbs, "train", dim)

In [813]:
# Check a value of the histogram and its bins
his_train_187

(array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 3, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [814]:
# Check value of the corresponding class label
cl_train_187

8

**Normalizing all the SIFT histograms by dividing with the max value**

In [817]:
# normalize histogram function

def hist_norm(bb_cnt, set_tag):
    """parameters: 
        bb_cnt: count of the input bbs
        set_tag: "train" or "val" or "test"
    """
    # for each bb, normalize the SIFT histogram by dividing the max value
    # use if else to avoid dividing by 0
    
    for bb in range(bb_cnt):
        
        if np.max(globals()['his_%s_%s' % (set_tag, bb)][0]) == 0:
            globals()['his_norm_%s_%s' % (set_tag, bb)] = \
              globals()['his_%s_%s' % (set_tag, bb)][0]
        else:
            globals()['his_norm_%s_%s' % (set_tag, bb)] = \
              globals()['his_%s_%s' % (set_tag, bb)][0]/np.max(globals()['his_%s_%s' % (set_tag, bb)][0])
        
    return

In [818]:
# normalizing train histograms
hist_norm(train_bbs, "train")

### Creating classification models
* Use the normalized SIFT histograms to train different linearly seperable (SVM and logistic regression) and simple L2 distance models
* Here we are going to use a One vs Rest classification model i.e. each class will be predicted with a yes or no when compared with all other classes

#### Training

In [819]:
# Get the normalized histograms and label data for the train set 
xtrain = [globals()['his_norm_train_%s' % bb] for bb in range(train_bbs)]
ytrain = [globals()['cl_train_%s' % bb] for bb in range(train_bbs)]

In [820]:
# Instantiate linear SVC and logistic regression models
lsvc = LinearSVC(penalty='l2', dual=False, multi_class = 'ovr', max_iter = 5000, C = 0.22, class_weight='balanced')
logovr = OneVsRestClassifier(LogisticRegression())

In [821]:
# Fit the training histograms and predict with SVC on training data itself
lsvc.fit(xtrain, ytrain)

y_train_score = lsvc.score(xtrain, ytrain)

print("SVC best score on Train set: {:.4f}".format(y_train_score))

SVC best score on Train set: 0.9311


In [828]:
# Fit the training histograms and predict with Logistic regression on training data itself 
logovr.fit(xtrain, ytrain)

y_train_pred = logovr.predict(xtrain)

print('LR Classifier best score on Train set: {:.4f}'.format(accuracy_score(ytrain, y_train_pred)))

LR Classifier best score on Train set: 0.9038


#### Validation

Check accuracy on validation set. Parameters were tuned using validation set performance.

In [823]:
# Function to build SIFT histogram from keypoint descriptors by grouping into clusters
def build_hist_val(desc, set_tag, dim, bb):
    """parameters:
        desc: keypoint descriptors variable for the input bb 
        set_tag: "train" or "val" or "test"
        dim: number of clusters used as bins
        bb: the index number of the bb from the list for which histogram is requested
    """
    
    # Use try except for handling bbs with no keypoints or descriptors
    try:
        # Run the bb keyponts through trained kmeans and assign clusters
        globals()['clst_%s_%s' % (set_tag, bb)] = kmeans.predict(desc)
    except:
        # Set histogram to zero if there are no keypoints
        globals()['his_%s_%s' % (set_tag, bb)] = (np.zeros((dim,),dtype=int),np.arange(0,dim+1))
    else:
        # Create histogram based on assigned clusters when descriptors are available for the bb
        globals()['his_%s_%s' % (set_tag, bb)] = np.histogram(globals()['clst_%s_%s' % (set_tag, bb)], \
                                                              bins=dim,range=(0,dim))
    
    return

In [824]:
# Build SIFT histogram for keypoints in each bb in the validation list
for bb in range(val_bbs):
    build_hist_val(globals()['dc_%s_%s' % ("val", bb)], "val", dim, bb)

In [825]:
# Normalize SIFT histograms from the validation bb list
for bb in range(val_bbs):
    
    # for each bb, normalize the SIFT histogram by dividing the max value
    # use if else to avoid dividing by 0
        
    if np.max(globals()['his_%s_%s' % ("val", bb)][0]) == 0:
            globals()['his_norm_%s_%s' % ("val", bb)] = \
              globals()['his_%s_%s' % ("val", bb)][0]
    else:
            globals()['his_norm_%s_%s' % ("val", bb)] = \
              globals()['his_%s_%s' % ("val", bb)][0]/np.max(globals()['his_%s_%s' % ("val", bb)][0])

In [827]:
# Get the normalized histograms and label data for the validation set 
xval = [globals()['his_norm_val_%s' % bb] for bb in range(val_bbs)]
yval = [globals()['cl_val_%s' % bb] for bb in range(val_bbs)]

In [830]:
# Predict with SVC on validation data
y_val_pred = lsvc.predict(xval)

print('SVC best score on Val set: {:.4f}'.format(accuracy_score(yval, y_val_pred)))

SVC best score on Val set: 0.7812


In [831]:
# Predict with SVC on validation data

y_val_pred = logovr.predict(xval)

print('LR Classifier best score on Val set: {:.2f}'.format(accuracy_score(yval, y_val_pred)))

LR Classifier best score on Val set: 0.75


Creating a third model to predict the class of the training SIFT histogram with the least L2 distance from validation sample

In [832]:
# Manual L2 distance check of SIFT histograms (Val vs Train)

# Create an empty list to store predictions
val_l2 = list()


# For each validation histogram check Euclidean distace with all training histograms and pick the least
for val in range(val_bbs):
    
    # Initialize the least distance and class label with first traing data
    lst_dist = distance.euclidean(xval[val],xtrain[0])
    cur_cls = yval[0]
    
    # Check the current validation histogram with every training histogram
    for train in range(train_bbs):
        
        # Calculate Euclidean distance
        cur_dist = distance.euclidean(xval[val],xtrain[train])
        
        # If the current distance is lower than the least one update least distance and current class
        if cur_dist < lst_dist:
            lst_dist = cur_dist
            cur_cls = ytrain[train]
    
    # Append the final class to prediction list 
    val_l2.append(cur_cls)

In [833]:
# Prediction score with manual L2 distance model on validation data

print('L2 dist Classifier best score on Val set: {:.4f}'.format(accuracy_score(yval, val_l2)))

L2 dist Classifier best score on Val set: 0.5596


### Section 2: Process using preprocessed and augmented data

In [274]:
# Load classes and augmented gray input images from json files
with open(base_dir+"/preproc_map_gray.json") as infile:
    gray_imgs = json.load(infile)

with open(base_dir+"/class_map_summary_gray.json") as infile:
    cls_summary = json.load(infile)

In [15]:
# Check a key and value
print(list(gray_imgs.items())[0])

print("\n",list(cls_summary.items())[0])

('000398_01_bb_cn_00_da_gray.jpg', {'image_source': '/images/apple/000398.jpg', 'bbox_source': '/yolofinalset/obj_train_data/000398.txt', 'preprocessing': ['bbox', 'cn', 'da', 'gray'], 'class': 'adidas', 'class_code': 0, 'set': 'train'})

 ('adidas', {'code': 0, 'num_bbs': 1528})


In [16]:
# Build a dataframe from class summary json
cls_summary = pd.DataFrame.from_dict(cls_summary)
cls_summary = cls_summary.T

In [36]:
cls_summary.head()

,code,num_bbs
adidas,0,1528
dhl,1,1528
apple,2,1528
bmw,3,1528
cocacola,4,1528


In [276]:
# Build a dataframe from gray pre-processed and augmented images json
gray_imgs = pd.DataFrame.from_dict(gray_imgs)
gray_imgs = gray_imgs.T

### Data assignment
* Get the list of tran, validation and test images from the dataframe created from input json

In [277]:
# Filter the train, validation and test file names based on the set name in the dataframe
train_set = gray_imgs[gray_imgs["set"]=="train"]
train_set = train_set.rename_axis('file').reset_index(level=0)
train_set = train_set.rename_axis('id').reset_index(level=0)

val_set = gray_imgs[gray_imgs["set"]=="val"]
val_set = val_set.rename_axis('file').reset_index(level=0)
val_set = val_set.rename_axis('id').reset_index(level=0)

test_set = gray_imgs[gray_imgs["set"]=="test"]
test_set = test_set.rename_axis('file').reset_index(level=0)
test_set = test_set.rename_axis('id').reset_index(level=0)

In [278]:
# Check a few rows in the train dataframe
train_set.head()

,id,file,image_source,bbox_source,preprocessing,class,class_code,set
0,0,000398_01_bb_cn_00_da_gray.jpg,/images/apple/000398.jpg,/yolofinalset/obj_train_data/000398.txt,"[bbox, cn, da, gray]",adidas,0,train
1,1,000398_01_bb_cn_01_da_gray.jpg,/images/apple/000398.jpg,/yolofinalset/obj_train_data/000398.txt,"[bbox, cn, da, gray]",adidas,0,train
2,2,000398_01_bb_cn_02_da_gray.jpg,/images/apple/000398.jpg,/yolofinalset/obj_train_data/000398.txt,"[bbox, cn, da, gray]",adidas,0,train
3,3,000398_01_bb_cn_03_da_gray.jpg,/images/apple/000398.jpg,/yolofinalset/obj_train_data/000398.txt,"[bbox, cn, da, gray]",adidas,0,train
4,4,000398_01_bb_cn_04_da_gray.jpg,/images/apple/000398.jpg,/yolofinalset/obj_train_data/000398.txt,"[bbox, cn, da, gray]",adidas,0,train


### Extract SIFT features from the training data and create histograms

In [280]:
# create SIFT feature extractor
sift_bb = cv2.SIFT_create()

In [281]:
# function to get keypoints and descripors for the given list of bbs
def key_desc_bb(file):
    """parameters: 
        file: input bb file name
        output: returns descriptors of keypoints
    """
    img_gray = plt.imread(gray_dir + file)
    
    kp, dc = sift_bb.detectAndCompute(img_gray, None)
    
    return dc

Below we extract all keypoints and descriptors for the three different bb lists

In [282]:
# Add a column to train dataframe and store all the keypoint descriptors for the corresponding files
train_set['kp_dc'] = train_set['file'].apply(key_desc_bb)

In [283]:
# Save the dataframe to pickle object for suture use
pkl_obj = open('train_set.pkl', 'wb')
pickle.dump(train_set,pkl_obj)
pkl_obj.close()

Now that we have extracted all the keypoints and descriptors from the training list, we need to stack them up. This will help us run k-means on it and cluster them into different groups.

In [284]:
# Stack all keypoint descriptors from all data skipping BBs with no keypoints
# This is to build clusters (BOW) and to build SIFT histogram
stack_desc = np.vstack(train_set[train_set['kp_dc'].apply(lambda x: x is not None)]['kp_dc'])

In [285]:
# Check shape of the stacked descriptors
stack_desc.shape

(1514473, 128)

In [286]:
# Check shape of the descriptors of one of the files
train_set['kp_dc'][1100].shape

(730, 128)

### K-means clustering
Run k-means algorithm on the extracted descriptors from the training set to form different clusters. Number of clusters is a hyper parameter arrived at after tuning.

In [287]:
# Define number of clusters/bins
dim = 1500

# Instantiate k-means
kmeans = KMeans(init="k-means++", n_clusters=dim, n_init=3)

# Fit all training keypoints and transform
dc_km = kmeans.fit_transform(stack_desc)

# Get cluster labels for the training keypoints
dc_kmy = kmeans.fit_predict(stack_desc)

In [288]:
# Check the shape of the descriptors after kmeans
# Number of features will be based on the bins
dc_km.shape

(1514473, 1500)

In [289]:
# Check the shape of the cluster labels for the training keypoint descriptors
dc_kmy.shape

(1514473,)

Building SIFT histogram for each bb based on the keypoint descriptor clusters

In [290]:
# Add a new column and store the created SIFT histogram
train_set["sift_hist"] = train_set[train_set['kp_dc'].apply(lambda x: x is not None)]['kp_dc']\
                        .apply(lambda x: kmeans.predict(x))\
                            .apply(lambda x: np.histogram(x,bins = 1500, range = (0,1500)))

In [291]:
# Save the updated dataframe with SIFT histogram as a pickle object
pkl_obj = open('train_set.pkl', 'wb')
pickle.dump(train_set,pkl_obj)
pkl_obj.close()

Normalize the SIFT histograms

In [292]:
# Perform normalization on the SIFT histograms and add it to a new column
train_set["norm_hist"] = train_set[train_set['sift_hist'].apply(lambda x: type(x) is tuple)]['sift_hist']\
                        .apply(lambda x: x[0]/np.max(x[0]))

### Training

In [293]:
# Get the normalized histograms and label data for the train set 
xtrain = list(train_set[train_set['sift_hist'].apply(lambda x: type(x) is tuple)]['norm_hist'])
ytrain = list(train_set[train_set['sift_hist'].apply(lambda x: type(x) is tuple)]['class_code'])

In [ ]:
#xtrain[xtrain.apply(lambda x: np.isnan(np.isnan(np.sum(x))))]

Train a SVC and a Logistic regression model

In [294]:
# Instantiate linear SVC and logistic regression models
lsvc = LinearSVC(penalty='l2', dual=False, multi_class = 'ovr', max_iter = 5000, C = 0.01, class_weight='balanced')
logovr = OneVsRestClassifier(LogisticRegression(penalty='l2', multi_class = 'ovr',\
                                                solver='liblinear',C = 0.1, max_iter = 5000))

In [295]:
# Fit the training histograms and predict with SVC on training data itself
lsvc.fit(xtrain, ytrain)

y_train_score = lsvc.score(xtrain, ytrain)

print("SVC best score on Train set: {:.4f}".format(y_train_score))

SVC best score on Train set: 0.8883


In [296]:
# Fit the training histograms and predict with Logistic regression on training data itself 
logovr.fit(xtrain, ytrain)

y_train_pred = logovr.predict(xtrain)

print('LR Classifier best score on Train set: {:.4f}'.format(accuracy_score(ytrain, y_train_pred)))

LR Classifier best score on Train set: 0.8839


In [297]:
# # Load classes and augmented gray input images from json files
# with open(base_dir+"/preproc_map_cn.json") as infile:
#     cn_imgs = json.load(infile)

# with open(base_dir+"/class_map_summary_cn.json") as infile:
#     cn_cls_summary = json.load(infile)

In [298]:
# cn_cls_summary = pd.DataFrame.from_dict(cn_cls_summary)
# cn_cls_summary = cn_cls_summary.T

In [299]:
# cn_imgs = pd.DataFrame.from_dict(cn_imgs)
# cn_imgs = cn_imgs.T

In [300]:
# val_set = cn_imgs[cn_imgs["set"]=="val"]
# val_set = val_set.rename_axis('file').reset_index(level=0)
# val_set = val_set.rename_axis('id').reset_index(level=0)

Perform the same set of steps on the validation files to extract keypoint descriptors, predict using the prevously trained kmeans model, build SIFT histogram, normalized and then predict using the previously trained SVC and Logistic Regression models. 

In [301]:
# function to get keypoints and descripors for the given list of bbs

def key_desc_bb_cn(file):
    """parameters: 
        file: input bb file name 
        output: returns keypoint descriptors
    """
    img_clr = plt.imread(base_dir + '/cn/' + file)
    gray = cv2.cvtColor(img_clr, cv2.COLOR_BGR2GRAY)
    
    kp, dc = sift_bb.detectAndCompute(gray, None)
    
    return dc

In [302]:
# Add a column to val dataframe and store all the keypoint descriptors for the corresponding files
val_set['kp_dc'] = val_set['file'].apply(key_desc_bb_cn)

In [303]:
# predict using the prevously trained kmeans model and assign clusters
val_set['kp_dc_cls'] = val_set[val_set['kp_dc'].apply(lambda x: x is not None)]['kp_dc']\
                            .apply(lambda x : kmeans.predict(x))

In [304]:
# Add a new column and store the created SIFT histogram
val_set["sift_hist"] = val_set[val_set['kp_dc'].apply(lambda x: x is not None)]['kp_dc_cls']\
                        .apply(lambda x: np.histogram(x,bins = 1500, range = (0,1500)))

In [305]:
# Perform normalization on the SIFT histograms and add it to a new column
val_set["norm_hist"] = val_set[val_set['sift_hist'].apply(lambda x: type(x) is tuple)]['sift_hist']\
                        .apply(lambda x: x[0]/np.max(x[0]))

In [306]:
# Get the normalized histograms and label data for the val set 
xval = list(val_set[val_set['sift_hist'].apply(lambda x: type(x) is tuple)]['norm_hist'])
yval = list(val_set[val_set['sift_hist'].apply(lambda x: type(x) is tuple)]['class_code'])

In [307]:
# Predict val set using previously trained SVC model

y_val_score = lsvc.score(xval, yval)

print("SVC best score on Train set: {:.4f}".format(y_val_score))

SVC best score on Train set: 0.7833


In [308]:
# Predict val set using previously trained Logistic Regression model

y_val_pred = logovr.predict(xval)

print('LR Classifier best score on Train set: {:.4f}'.format(accuracy_score(yval, y_val_pred)))

LR Classifier best score on Train set: 0.7917


In [ ]:
# Save the validation dataframe
pkl_obj = open('val_set.pkl', 'wb')
pickle.dump(val_set,pkl_obj)
pkl_obj.close()